# Bài 1

### Dựa trên bộ dữ liệu tự thu thập từ các trang báo mạng Tiếng Việt (5 lớp/ mỗi lớp 10 mẩu tin), hãy thực hiện các yêu cầu sau:

Trang báo lựa chọn sẽ cào là 24h.com.vn 

Chia ra 5 label như sau:
- Giao Thông : https://www.24h.com.vn/tai-nan-giao-thong-c408.html
- Chính trị - Xã hội : https://www.24h.com.vn/chinh-tri-xa-hoi-c981.html
- Kinh doanh : https://www.24h.com.vn/kinh-doanh-c161.html
- Giải trí : https://www.24h.com.vn/giai-tri-c731.html
- Thể thao : https://www.24h.com.vn/bong-da-c48.html

In [2]:
# !pip install undetected-chromedriver
# !pip install selenium

In [1]:
web_giaothong = 'https://www.24h.com.vn/tai-nan-giao-thong-c408.html'
web_chinhtri = 'https://www.24h.com.vn/chinh-tri-xa-hoi-c981.html'
web_thoitiet = 'https://www.24h.com.vn/du-bao-thoi-tiet-c568.html'
web_thethao = 'https://www.24h.com.vn/the-thao-c101.html'
web_giaitri = 'https://www.24h.com.vn/giai-tri-c731.html'

all_webs = [web_giaothong, web_chinhtri,web_thoitiet, web_thethao, web_giaitri]
labels = ['giaothong', 'chinhtri', 'kinhdoanh', 'thethao', 'giaitri']


In [2]:
import undetected_chromedriver as uc 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np

In [3]:

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
wait = WebDriverWait(driver,20)


In [4]:
def get_content(link):
    driver.get(link)
    content= wait.until(EC.presence_of_all_elements_located((By.XPATH,'//article/p')))
    title=driver.title.replace(u'\xa0', u' ')
    content = [p.text for p in content if p.text != '']
    return title,content


def get_all_content_label(link_labels,label):
    driver.get(link_labels)
    all_links = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//header/*[@class="cate-24h-foot-home-latest-list__name"]/a')))
    while len(all_links) < 10:
        wait.until(EC.presence_of_element_located((By.XPATH,'//div/p/a'))).click()
        sleep(2)
        all_links = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//header/*[@class="cate-24h-foot-home-latest-list__name"]/a')))

    all_links = [link.get_attribute('href') for link in all_links]
    
    titles = []
    contents = []
    labels = []
    for link in tqdm(all_links):
        title, content = get_content(link)
        titles.append(title)
        contents.append(content)
        labels.append(label)

    return titles, contents, labels    

big_tit= []
big_cont=[]
big_labels=[]
df = pd.DataFrame(columns=['title','content','label'])
for links,label in zip(all_webs,labels):
    titles, contents, all_labels = get_all_content_label(links,label)
    big_tit += titles
    big_cont += contents
    big_labels += all_labels

df = pd.DataFrame({'title':big_tit,'content':big_cont,'label':big_labels})
df.to_csv('data.csv',index=False)
df.sample(6)

100%|██████████| 14/14 [00:56<00:00,  4.01s/it]


,title,content,label
76,Con gái rapper Thái VG gây sốt khi lên sóng tr...,[Chỉ xuất hiện vỏn vẹn 6 giây trong trailer ch...,giaitri
53,Tin tức 24h qua: Tòa bác đơn xin hoãn chấp hàn...,"[Tòa bác đơn xin hoãn chấp hành án trong vụ ""E...",kinhdoanh
77,"Quán quân Rap Việt mùa 3: ""Người miền núi chất...",[Đêm chung kết Công bố và Trao giải Rap Việt m...,giaitri
81,"Dàn mỹ nhân ""Cổng mặt trời"" hội ngộ: Nhan sắc ...",[Cổng mặt trời là bộ phim truyền hình gắn liền...,giaitri
31,"Sau khi bị khởi tố, nguyên Giám đốc Sở GD-ĐT G...","[Ngày 8-9, UBKT Tỉnh ủy Gia Lai cho biết đã đề...",chinhtri
14,Quảng Bình: Hơn 1.600 vụ ly hôn trong 8 tháng,"[Ngày 9-9, ông Nguyễn Hữu Tuyến, Chánh án TAND...",giaothong


In [5]:
driver.quit()

### 1.Tiền xử lí dữ liệu

In [8]:
import re
from underthesea import sent_tokenize,word_tokenize
import re
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
from nltk.util import ngrams
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
df = pd.read_csv('data.csv', encoding='utf-8')
df.head(5)

,title,content,label
0,"Clip: Thấy con chó lao thẳng vào xe máy SH, tà...",['Hình ảnh vụ tai nạn được chia sẻ trên nhóm F...,giaothong
1,"Đau đầu với 650 chiếc xe máy ""bỏ quên"" tại nhà...",['Hàng trăm xe máy để tại tầng 3-5 của Nhà để ...,giaothong
2,"Ngông nghênh đi lên Vành đai 2 trên cao, cả lo...","['Tuyến Vành đai 2 trên, đoạn lối xuống đường ...",giaothong
3,Xe tải bị lật nghiêng sau tai nạn trên cao tốc...,"['Hiện trường vụ tai nạn.', 'Chiều 8/9, đại di...",giaothong
4,"Clip: Phóng xe máy như bay lao vào đầu ô tô, đ...",['Hình ảnh vụ tai nạn hôm 5/9 được chia sẻ trê...,giaothong


In [10]:
def processcing_text(text):
    tweet = text
    #lower
    tweet = tweet.lower()
    #convert any url link to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet)
    #convert any @Username to AT_USER
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet)

    #Remove not alphanumeric symbols white spaces
    tweet = re.sub(r'[^\w]',' ', tweet)
    #Removes # hashtag in front of a word
    tweet = re.sub(r'#([\w]+)', r'\1', tweet)
    tweet = re.sub(r'#([^\s]+)',r'\1',tweet)
    #remove :( or :)
    tweet = tweet.replace(':)','')
    tweet = tweet.replace(':(','')
    #remove numbers
    tweet = ''.join([i for i in tweet if not i.isdigit()])
    #remove multiple exclamation
    tweet = re.sub(r'(!)\1+', ' ', tweet)
    #remove multiple question marks
    tweet = re.sub(r'(\?)\1+','', tweet)
    #remove multistop
    tweet = re.sub(r'(\.)\1+','', tweet)
    #Remove additional whitespace
    tweet = re.sub(r'[\s]+',' ', tweet)
    tweet = re.sub(r'[\n]+',' ', tweet)
    #lemma 
    # tweet = " ".join([Word(word).lemmatize() for word in tweet.split()])
    #stemmer
    # st=  PorterStemmer()
    # tweet = " ".join([st.stem(word) for word in tweet.split()])
    #remove emoteicon from text
    # tweet = re.sub('')
    row = tweet
    return row

df['content']=df['content'].apply(processcing_text)
df.head(1)['content'].values[0]

' hình ảnh vụ tai nạn được chia sẻ trên nhóm facebook về giao thông cho thấy sau tai nạn tài xế xe máy sh có thể ngồi dậy và may mắn không va chạm với phương tiện nào khác trong khi đó con chó gây tai nạn bỏ chạy từ vụ tai nạn trên trao đổi với pv về quy định xử lý hành vi thả rông động vật gây tai nạn giao thông luật sư lê văn kiên trưởng văn phòng luật sư ánh sáng công lý cho biết việc xử lý hành vi thả rông vật nuôi được quy định tại nghị định nđ cp nghị định quy định xử phạt vi phạm hành chính trong lĩnh vực an ninh trật tự an toàn xã hội phòng chống tệ nạn xã hội phòng cháy chữa cháy cứu nạn cứu hộ phòng chống bạo lực gia đình theo nghị định nđ cp quy định chủ vật nuôi sẽ bị phạt tiền từ đồng đến đồng về hành vi để động vật nuôi gây thương tích hoặc gây thiệt hại tài sản cho tổ chức cá nhân khác nhưng không bị truy cứu trách nhiệm hình sự với hành vi để vật nuôi xâm lấn lòng đường vỉa hè vườn hoa sân chơi đô thị nơi sinh hoạt chung trong khu dân cư khu đô thị chủ vật nuôi sẽ bị xử

### 2. Tách từ (Tokenize) sử dụng thư viện pyvi hay underthesea


In [11]:
from underthesea import word_tokenize


def toke(text):
    t=word_tokenize(text, format='text')
    # t=word_tokenize(t)
    return t

df['content']=df['content'].apply(toke)

df['content'].head(1).values[0]

'hình_ảnh vụ tai_nạn được chia_sẻ trên nhóm facebook về giao_thông cho thấy sau tai_nạn tài_xế xe_máy sh có_thể ngồi dậy và may_mắn không va_chạm với phương_tiện nào khác trong khi đó con chó gây tai_nạn bỏ chạy từ vụ tai_nạn trên trao_đổi với pv về quy_định xử_lý hành_vi thả rông động_vật gây tai_nạn giao_thông luật_sư lê_văn_kiên_trưởng văn_phòng luật_sư ánh_sáng công_lý cho biết việc xử_lý hành_vi thả rông vật_nuôi được quy_định tại nghị_định nđ cp nghị_định quy_định xử_phạt vi_phạm hành_chính trong lĩnh_vực an_ninh trật_tự an_toàn xã_hội phòng_chống tệ_nạn xã_hội phòng cháy chữa_cháy cứu nạn cứu_hộ phòng_chống bạo_lực gia_đình theo nghị_định nđ cp quy_định chủ vật_nuôi sẽ bị phạt tiền từ đồng đến đồng về hành_vi để động_vật_nuôi gây thương_tích hoặc gây thiệt_hại tài_sản cho tổ_chức cá_nhân khác nhưng không bị truy_cứu trách_nhiệm hình_sự với hành_vi để vật_nuôi xâm_lấn lòng_đường vỉa_hè vườn hoa sân_chơi đô_thị nơi sinh_hoạt chung trong khu dân_cư khu đô_thị chủ vật_nuôi sẽ bị xử_

### 3. Trích xuất đặc trưng TF-IDF bằng thư viện sklearn


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df['content'].values
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(corpus)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
X.toarray().shape, y.shape

((88, 3737), (88,))

Label encoder:
- Chính trị : 0
- Giải trí : 1
- Giao thông : 2
- Kinh doanh : 3
- Thể thao : 4

In [13]:
label_encoder.inverse_transform([0,1,2,3,4])

array(['chinhtri', 'giaitri', 'giaothong', 'kinhdoanh', 'thethao'],
      dtype=object)

### 4. Đánh giá bộ dữ liệu với giải thuật KNN bằng phương pháp 5-Fold (k-fold)


In [14]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=2)  

kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_val)

    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print(f'Average Accuracy: {average_accuracy}')



Average Accuracy: 0.661437908496732


### 5. Huấn luyện dữ liệu cho bài toán phân loại văn bản với tỷ lệ dữ liệu 8:2 (8 phần train, 2 phần test) sử dụng đặc trưng TF-IDF và 2 giải thuật bayes và SVM

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()
nb_classifier.fit(X_train.toarray() , y_train)
nb_predictions = nb_classifier.predict(X_test.toarray())


In [16]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='linear')

svm_classifier.fit(X_train, y_train)

svm_predictions = svm_classifier.predict(X_test)


### 6. Tính độ đo F1 score


In [17]:
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix 

print('F1 score of Naive Bayes: ', f1_score(y_test, nb_predictions, average='micro'))
print('F1 score of SVM: ', f1_score(y_test, svm_predictions, average='micro'))


F1 score of Naive Bayes:  0.7222222222222222
F1 score of SVM:  0.5555555555555556


### 7. Tính độ đo Accuracy


In [18]:
print('Accuracy score of Naive Bayes: ', accuracy_score(y_test, nb_predictions))
print('Accuracy score of SVM: ', accuracy_score(y_test, svm_predictions))

Accuracy score of Naive Bayes:  0.7222222222222222
Accuracy score of SVM:  0.5555555555555556


### 8. Tính độ đo Confusion Matrix

In [19]:
print('Confusion matrix of Naive Bayes: \n', confusion_matrix(y_test, nb_predictions)) 
print('Confusion matrix of SVM: \n', confusion_matrix(y_test, svm_predictions))

Confusion matrix of Naive Bayes: 
 [[2 0 1 0 0]
 [0 4 0 0 1]
 [1 1 3 0 0]
 [0 0 1 0 0]
 [0 0 0 0 4]]
Confusion matrix of SVM: 
 [[3 0 0 0 0]
 [1 3 0 0 1]
 [3 0 2 0 0]
 [0 0 0 1 0]
 [1 0 2 0 1]]


### 9. So sánh kết quả các độ đo 6,7,8 với 2 giải thuật học máy ở trên


So sánh giải thuật SVM và Bayes thì Bayes cho kết quả tốt, vượt trội hơn SVM 

Vì lí do dữ liệu vẫn còn khá ít và các label phân chia nhau chưa rõ ràng nên điểm số chưa cao 

Cần khắc phục bằng cách tăng dữ liệu và phân chia rõ ràng hơn


### 10.Lưu model với giải thuật đạt kết quả tốt nhất


In [20]:
from joblib import dump, load

dump(nb_classifier, 'nb_classifier.pkl')
dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [21]:
loaded_nb_classifier = load('nb_classifier.pkl')
loaded_tfidf_vectorizer = load('tfidf_vectorizer.pkl')

In [22]:
def predict_text_to_label(text):
    text = processcing_text(text)
    text = toke(text)
    text = loaded_tfidf_vectorizer.transform([text]).toarray()
    label = loaded_nb_classifier.predict(text)
    label_name = label_encoder.inverse_transform(label)
    return label_name[0]

text_test='Một vụ tai nạn giao thông thảm khốc xảy ra ở tân bình'
predict_text_to_label(text_test)

'giaothong'

## Bài 2: Cho ví dụ sử dụng HashVectorizer như sau:


```
from sklearn.feature_extraction.text import HashingVectorizer
corpus = [
 ' Hôm_nay tôi đi_học',
 ' Hôm_nay tôi đi_học ở trường',
 ' Hôm_nay tôi nghỉ ở nhà',
 ' Hôm_nay tôi có đi_học không?',
]
vectorizer = HashingVectorizer(n_features=2**4)
X = vectorizer.fit_transform(corpus)
print(X.shape)
```

In [25]:
from sklearn.feature_extraction.text import HashingVectorizer
corpus = [
 ' Hôm_nay tôi đi_học',
 ' Hôm_nay tôi đi_học ở trường',
 ' Hôm_nay tôi nghỉ ở nhà',
 ' Hôm_nay tôi có đi_học không?',
]
vectorizer = HashingVectorizer(n_features=2**5)
X = vectorizer.fit_transform(corpus)
print(X.shape)


(4, 32)


Hàm HashingVectorizer sẽ tương tự tfidf nhưng số lượng feature thay vì đưa vô sẽ dễ ra một vecto thưa(Spare vector) thì sẽ được đưa vào một mảng băm với số lượng feature là tùy chỉnh

### Bài 3: Sử dụng HashVectorizer thay cho đặc trưng TF-IDF ở bài 1

In [39]:
hash_vec = HashingVectorizer(n_features=2**14)
corpus = df['content'].values
X = hash_vec.fit_transform(corpus)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
X.toarray().shape, y.shape

((88, 16384), (88,))

In [40]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()
nb_classifier.fit(X_train.toarray() , y_train)
nb_predictions = nb_classifier.predict(X_test.toarray())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='linear')

svm_classifier.fit(X_train, y_train)

svm_predictions = svm_classifier.predict(X_test)


print('F1 score of Naive Bayes: ', f1_score(y_test, nb_predictions, average='micro'))
print('F1 score of SVM: ', f1_score(y_test, svm_predictions, average='micro'))
print('Accuracy score of Naive Bayes: ', accuracy_score(y_test, nb_predictions))
print('Accuracy score of SVM: ', accuracy_score(y_test, svm_predictions))
print('Confusion matrix of Naive Bayes: \n', confusion_matrix(y_test, nb_predictions)) 
print('Confusion matrix of SVM: \n', confusion_matrix(y_test, svm_predictions))

F1 score of Naive Bayes:  0.7222222222222222
F1 score of SVM:  0.6666666666666666
Accuracy score of Naive Bayes:  0.7222222222222222
Accuracy score of SVM:  0.6666666666666666
Confusion matrix of Naive Bayes: 
 [[2 0 1 0 0]
 [0 4 0 0 1]
 [1 1 3 0 0]
 [0 0 1 0 0]
 [0 0 0 0 4]]
Confusion matrix of SVM: 
 [[3 0 0 0 0]
 [0 4 0 0 1]
 [3 0 2 0 0]
 [0 0 0 1 0]
 [0 1 1 0 2]]
